IMPORTING LIBRARIES

In [2]:
#import relevant libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from sqlalchemy import create_engine
from scipy import stats


LOADING MOVIE BUDGET DATASET

In [3]:
Movie_budgets = pd.read_csv("../Data/tn.movie_budgets.csv", low_memory=False)
Movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [4]:
#check missing values
Movie_budgets.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [5]:
#Check duplicates
Movie_budgets.duplicated().sum()

0

In [6]:
print(Movie_budgets.columns)
print(Movie_budgets.info())
print(Movie_budgets.isna().sum().sort_values(ascending=False))
print(Movie_budgets.duplicated().sum())

Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB
None
id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64
0


In [7]:
# Convert release_date to datetime
Movie_budgets['release_date'] = pd.to_datetime(Movie_budgets['release_date'], errors='coerce')

# Extract release year
Movie_budgets['release_year'] = Movie_budgets['release_date'].dt.year

# Clean movie title (lowercase, strip spaces)
Movie_budgets['movie_clean'] = Movie_budgets['movie'].str.lower().str.strip()


In [8]:
#drop the $ Dollar sign and , from production_budget, domestic_gross and worldwide_gross columns
Movie_budgets['production_budget'] = Movie_budgets['production_budget'].str.replace('$','').str.replace(',','')
Movie_budgets['domestic_gross'] = Movie_budgets['domestic_gross'].str.replace('$','').str.replace(',','')
Movie_budgets['worldwide_gross'] = Movie_budgets['worldwide_gross'].str.replace('$','').str.replace(',','')

In [9]:
#changing tatetime format
Movie_budgets['release_date'] = pd.to_datetime(Movie_budgets['release_date'], errors='coerce')

In [10]:
#Changing the production budget, domestic gross and worldwide gross  to numeric
Movie_budgets['production_budget'] = pd.to_numeric(Movie_budgets['production_budget'], errors='coerce')
Movie_budgets['domestic_gross'] = pd.to_numeric(Movie_budgets['domestic_gross'], errors='coerce')
Movie_budgets['worldwide_gross'] = pd.to_numeric(Movie_budgets['worldwide_gross'], errors='coerce')

In [11]:
# Create profit column to determine net earnings by subtracting production costs from total revenues.

Movie_budgets['profit'] = Movie_budgets['worldwide_gross'] - Movie_budgets['production_budget']

In [12]:
Movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,movie_clean,profit
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,avatar,2351345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,pirates of the caribbean: on stranger tides,635063875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019,dark phoenix,-200237650
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,avengers: age of ultron,1072413963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,star wars ep. viii: the last jedi,999721747


In [13]:
Movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   release_year       5782 non-null   int32         
 7   movie_clean        5782 non-null   object        
 8   profit             5782 non-null   int64         
dtypes: datetime64[ns](1), int32(1), int64(5), object(2)
memory usage: 384.1+ KB


LOADING TMDB MOVIES

In [14]:
Movies_tmdb = pd.read_csv("../Data/tmdb.movies.csv", low_memory=False)
Movies_tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [15]:
Movies_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [16]:
print(Movies_tmdb.columns)
print(Movies_tmdb.info())
print(Movies_tmdb.isna().sum().sort_values(ascending=False))
print(Movies_tmdb.duplicated().sum())

Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB
None
Unnamed: 0           0
genre_ids            0
id                   0
o

In [17]:
#release_date column to datetime format
Movies_tmdb['release_date'] = pd.to_datetime(Movies_tmdb['release_date'], errors='coerce')

In [18]:
# drop the unnamed column
Movies_tmdb = Movies_tmdb.drop(columns=['Unnamed: 0'])

In [19]:
# Convert release_date to datetime
Movies_tmdb['release_date'] = pd.to_datetime(Movies_tmdb['release_date'], errors='coerce')

# Extract year
Movies_tmdb['year'] = Movies_tmdb['release_date'].dt.year

# Clean title
Movies_tmdb['title_clean'] = Movies_tmdb['title'].str.lower().str.strip()


In [20]:
Movies_tmdb.isna().sum()

genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
year                 0
title_clean          0
dtype: int64

In [21]:
Movies_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   genre_ids          26517 non-null  object        
 1   id                 26517 non-null  int64         
 2   original_language  26517 non-null  object        
 3   original_title     26517 non-null  object        
 4   popularity         26517 non-null  float64       
 5   release_date       26517 non-null  datetime64[ns]
 6   title              26517 non-null  object        
 7   vote_average       26517 non-null  float64       
 8   vote_count         26517 non-null  int64         
 9   year               26517 non-null  int32         
 10  title_clean        26517 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(1), int64(2), object(5)
memory usage: 2.1+ MB


In [22]:
Movies_tmdb.duplicated().sum()

1020

LOADING MOVIE GROSS DATASET

In [23]:
Movie_bom = pd.read_csv('../Data/bom.movie_gross.csv', low_memory=False)
Movie_bom.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [24]:
Movie_bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [26]:
# Convert foreign_gross to numeric
Movie_bom['foreign_gross'] = pd.to_numeric(Movie_bom['foreign_gross'], errors='coerce')

# Drop rows missing both gross values
Movie_bom= Movie_bom.dropna(subset=['domestic_gross'], how='all')

# Fill missing foreign_gross with median (preserve 70%+)
median_foreign = Movie_bom['foreign_gross'].median()
Movie_bom['foreign_gross'].fillna(median_foreign, inplace=True)

# Fill missing studio with "Unknown"
Movie_bom['studio'].fillna("Unknown", inplace=True)

# Create a clean title column
Movie_bom['title_clean'] = Movie_bom['title'].str.lower().str.strip()

#change year to datetime
Movie_bom['year']=pd.to_datetime(Movie_bom['year'])

Movie_bom['worldwide_gross']=Movie_bom['foreign_gross']+Movie_bom['foreign_gross']

In [27]:
Movie_bom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3359 entries, 0 to 3386
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   title            3359 non-null   object        
 1   studio           3359 non-null   object        
 2   domestic_gross   3359 non-null   float64       
 3   foreign_gross    3359 non-null   float64       
 4   year             3359 non-null   datetime64[ns]
 5   title_clean      3359 non-null   object        
 6   worldwide_gross  3359 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 209.9+ KB


LOADING THE IM.DB DATABASE

In [28]:
# Connect to the imdb-style database
conn = sqlite3.connect("../Data/im.db")

# List all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in im.db:")
print(tables)

Tables in im.db:
            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers
